In [10]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [11]:
import json
from pprint import pprint

from utils import csv_utils, utils
from clients import astroquery_client

In [12]:
collection_name = 'SLACS'

data_path = '/data/bwedig/roman-pandeia/data'

Pull in targets from CSV

In [13]:
mast_csv_filepath = os.path.join(repo_path, 'data', collection_name, 'HST_2023-11-07T13_09_40-06_00.csv')
dataset_list = utils.deserialize(mast_csv_filepath)

print('Number of targets retrieved from CSV: ' + str(len(dataset_list)))

Number of targets retrieved from CSV: 248


Get appropriate data URIs so they can be downloaded

In [14]:
data_uri_list, dataset_list = astroquery_client.retrieve_datasets(dataset_list)

data_uri_cache_filepath = os.path.join(repo_path, 'data', 'cache', 'data_uri_cache_' + collection_name + '.txt')

with open(data_uri_cache_filepath, 'w') as f:
    f.write(json.dumps(data_uri_list))

 32%|███▏      | 79/248 [00:32<01:09,  2.44it/s]/nfshome/bwedig/roman-pandeia/clients/astroquery_client.py:47: UserWarning: No matching dataset found for U9OP6301M
  warnings.warn(message)
 32%|███▏      | 80/248 [00:32<01:06,  2.53it/s]/nfshome/bwedig/roman-pandeia/clients/astroquery_client.py:47: UserWarning: No matching dataset found for U9OP6302M
  warnings.warn(message)
 33%|███▎      | 81/248 [00:33<01:06,  2.53it/s]/nfshome/bwedig/roman-pandeia/clients/astroquery_client.py:47: UserWarning: No matching dataset found for U9OP6303M
  warnings.warn(message)
 33%|███▎      | 82/248 [00:33<01:08,  2.42it/s]/nfshome/bwedig/roman-pandeia/clients/astroquery_client.py:47: UserWarning: No matching dataset found for U9OP6304M
  warnings.warn(message)
 50%|████▉     | 123/248 [00:50<00:51,  2.44it/s]/nfshome/bwedig/roman-pandeia/clients/astroquery_client.py:47: UserWarning: No matching dataset found for U9OP5101M
  warnings.warn(message)
 50%|█████     | 124/248 [00:50<00:49,  2.52it/s]/nfsh



Skipped: 56
[]


Multiple: 0
[]


Download all

In [15]:
download_path = os.path.join(data_path, 'mastDownload', collection_name)

with open(data_uri_cache_filepath) as f:
    data = f.read()

data_uri_list = json.loads(data)

astroquery_client.download_datasets(data_uri_list, download_path)

# add fits filepaths to datasets
for dataset in dataset_list:
    if dataset.filename is not None:
        fits_filepath = os.path.join(download_path, dataset.filename)
        dataset.set_fits_filepath(fits_filepath)

Ready to download 192 files.

Dataset 1 of 192: mast:HST/product/j93i30itq_drz.fits
('COMPLETE', None, None)
Dataset 2 of 192: mast:HST/product/j93i30iuq_drz.fits
('COMPLETE', None, None)
Dataset 3 of 192: mast:HST/product/j93i38zlq_drz.fits
('COMPLETE', None, None)
Dataset 4 of 192: mast:HST/product/j93i38zmq_drz.fits
('COMPLETE', None, None)
Dataset 5 of 192: mast:HST/product/j93i33h0q_drz.fits
('COMPLETE', None, None)
Dataset 6 of 192: mast:HST/product/j93i33h1q_drz.fits
('COMPLETE', None, None)
Dataset 7 of 192: mast:HST/product/j93i12fpq_drz.fits
('COMPLETE', None, None)
Dataset 8 of 192: mast:HST/product/j93i12fqq_drz.fits
('COMPLETE', None, None)
Dataset 9 of 192: mast:HST/product/j93i49ojq_drz.fits
('COMPLETE', None, None)
Dataset 10 of 192: mast:HST/product/j93i49okq_drz.fits
('COMPLETE', None, None)
Dataset 11 of 192: mast:HST/product/j93i41dmq_drz.fits
('COMPLETE', None, None)
Dataset 12 of 192: mast:HST/product/j93i41dnq_drz.fits
('COMPLETE', None, None)
Dataset 13 of 192: 

In [16]:
from pprint import pprint

pprint(dataset_list[0].__dict__)

{'actual_duration': '420.0',
 'aper': 'WFC1',
 'central_wavelength': '4329.1958',
 'cutout': None,
 'cutout_filepath': None,
 'data_set_name': 'J93I30ITQ',
 'data_uri': 'mast:HST/product/j93i30itq_drz.fits',
 'dec': '-0.5881777777778',
 'filename': 'j93i30itq_drz.fits',
 'fits_filepath': '/data/bwedig/roman-pandeia/data/mastDownload/SLACS/j93i30itq_drz.fits',
 'instrument': 'ACS   ',
 'peak_value': None,
 'preview_name': 'J93I30ITQ',
 'ra': '156.463875',
 'release_date': '2005-04-25T01:10:56',
 'spec': 'CLEAR1L;F435W',
 'start_time': '2004-10-23T19:35:48.977000',
 'stop_time': '2004-10-23T19:42:50.023000',
 'target_name': '0272-51941-151'}


Write updated dataset list to a CSV, ignoring any datasets without images

In [17]:
from utils import csv_utils

dataset_list = [f for f in dataset_list if f.fits_filepath is not None]

csv_filepath = os.path.join(repo_path, 'data', collection_name, collection_name + '.csv')

csv_utils.dataset_list_to_csv(dataset_list, csv_filepath)